In [ ]:
import math
import numpy as np
from numba import cuda, jit, float64
import time
from datetime import datetime
N = 128                        # 128/256/512/1024
A = np.random.random((N, N))   #Матрицы для умножения
B = np.random.random((N, N))
C = np.zeros((N, N))           # создание матрицы "С" - "матрица результат"

In [ ]:
def cpu_MM(A, B, C):
    for i in range(C.shape[0]):
        for j in range(C.shape[1]):                     # Цикл перемножения матриц
            summation = 0
            for k in range(A.shape[1]):
                summation += A[i, k] * B[k, j]  #Перемножение очередных элементов 
            C[i, j] = summation                 #Запись в ячейку выходной матрицы

start = datetime.now()        # Для замера времени
cpu_MM(A, B, C) 
print('MatMul на CPU -', datetime.now() - start)
host_mat_1 = C                #Для проверки на корректность умножения 


MatMul на CPU - 0:00:01.271411


In [ ]:
@jit
def cpu_MMJ(A, B, C):
    for i in range(C.shape[0]):
        for j in range(C.shape[1]):
            summation = 0
            for k in range(A.shape[1]):
                summation += A[i, k] * B[k, j]
            C[i, j] = summation
start = datetime.now()        # Для замера времени
cpu_MMJ(A, B, C) 
print('CPU MatMul c применением компилятора numba - jit', datetime.now() - start)
host_mat_2 = C              #Для проверки на корректность умножения 


CPU MatMul c применением компилятора numba - jit 0:00:00.210743


In [ ]:
@cuda.jit
def mat_mul_naive_kernal(A, B, C):
    i, j = cuda.grid(2)                       # Эта строчка говорит что i и j - индексы двумерной сети куды
    if i < C.shape[0] and j < C.shape[1]:
        summation = 0
        for k in range(A.shape[1]):
            summation += A[i, k] * B[k, j]
        C[i, j] = summation


TPB = 16
def host_naive(A, B, C):

    d_A = cuda.to_device(A)                     # Выделяем куда ядра под значения матриц
    d_B = cuda.to_device(B)
    d_C = cuda.device_array(C.shape, np.float64)  
    #"Вычисление размера сети"
    threadsperblock = (TPB, TPB)            # Размер блока 16*16 потоков             
    blockspergrid_x = math.ceil(A.shape[0] / threadsperblock[0])      # делю размер матрицы "А" на размер блока и получаю количество блоков на все потоки по Х
    blockspergrid_y = math.ceil(B.shape[1] / threadsperblock[1])        # по Y матрицу"В"
    blockspergrid = (blockspergrid_x, blockspergrid_y)        # Указываю вычесленые количество блоков для указания размеров сети Х на Y

    mat_mul_naive_kernal[blockspergrid, threadsperblock](d_A, d_B, d_C)       #

    return d_C.copy_to_host()
start = datetime.now()        # Для замера времени
ans = host_naive(A, B, C)
print('GPU MM с cuda.jit:', datetime.now() - start)
dev_mat = C       # Для проверки на корректность умножения 

GPU MM с cuda.jit: 0:00:00.218543


In [ ]:
print(np.allclose(dev_mat, host_mat_2)) # Проверка корректного умножения на ЦПУ И ГПУ, True = входные значения в функцию np равны
print(np.allclose(dev_mat, host_mat_1))

True
True
